# 1. Initializations

## 1.1 General imports

In [1]:
### data management
import pandas as pd
import numpy as np
# import statsmodels.api as sm

### graphical plotly basics
import plotly.graph_objects as go
import plotly.express as px
# for jupyter notebook display management
import plotly.io as pio
pio.renderers.default = "notebook"

## 1.2 General dataframe functions

In [2]:
import smartcheck.dataframe_common as dfc

# 2. Loading and Data Quality

## 2.1 Loading of data sets and general exploration

### 2.2.1 VELIB DISPO

In [3]:
df_disp_velib_raw = dfc.load_dataset_from_config('velib_dispo_data', sep=';')

if df_disp_velib_raw is not None and isinstance(df_disp_velib_raw, pd.DataFrame):
    display(df_disp_velib_raw.head())
    dfc.log_general_info(df_disp_velib_raw)
    nb_first, nb_total = dfc.detect_and_log_duplicates_and_missing(df_disp_velib_raw)
    if nb_first != nb_total:
        print(dfc.duplicates_index_map(df_disp_velib_raw))
    df_disp_velib = dfc.normalize_column_names(df_disp_velib_raw)

[INFO]-[2025-06-01 09:52:12,448] File path resolved from configuration : https://drive.google.com/file/d/1kBkBYm55NrNZ55fjD2G7jvdJqIz517Qm/view?usp=drive_link.
[INFO]-[2025-06-01 09:52:12,448] File ID extracted from URL: 1kBkBYm55NrNZ55fjD2G7jvdJqIz517Qm


,Identifiant station,Nom station,Station en fonctionnement,Capacité de la station,Nombre bornettes libres,Nombre total vélos disponibles,Vélos mécaniques disponibles,Vélos électriques disponibles,Borne de paiement disponible,Retour vélib possible,Actualisation de la donnée,Coordonnées géographiques,Nom communes équipées,Code INSEE communes équipées,station_opening_hours
0,16107,Benjamin Godard - Victor Hugo,OUI,0,27,8,0,8,OUI,OUI,2025-04-17T15:39:50+00:00,"48.865983, 2.275725",Paris,75056,NaN
1,44015,Rouget de L'isle - Watteau,OUI,0,12,8,4,4,OUI,OUI,2025-04-17T15:37:57+00:00,"48.778192750803, 2.3963020229163",Vitry-sur-Seine,94081,NaN
2,40001,Hôpital Mondor,OUI,0,23,6,2,4,OUI,OUI,2025-04-17T15:37:02+00:00,"48.798922410229, 2.4537451531298",Créteil,94028,NaN
3,9020,Toudouze - Clauzel,OUI,0,18,2,0,2,OUI,OUI,2025-04-17T15:41:24+00:00,"48.87929591733507, 2.3373600840568547",Paris,75056,NaN
4,13007,Le Brun - Gobelins,OUI,0,37,10,6,4,OUI,OUI,2025-04-17T15:38:07+00:00,"48.835092787824, 2.3534681351338",Paris,75056,NaN


[INFO]-[2025-06-01 09:52:15,062] Dataset shape: 1472 rows x 15 columns
[INFO]-[2025-06-01 09:52:15,062] For quantitative variable description use:
df.select_dtypes(include=np.number).describe()
[INFO]-[2025-06-01 09:52:15,062] For quantitative correlation matrix use:
df.select_dtypes(include=np.number).corr()
[INFO]-[2025-06-01 09:52:15,062] DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1472 entries, 0 to 1471
Data columns (total 15 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Identifiant station             1472 non-null   object 
 1   Nom station                     1472 non-null   object 
 2   Station en fonctionnement       1472 non-null   object 
 3   Capacité de la station          1472 non-null   int64  
 4   Nombre bornettes libres         1472 non-null   int64  
 5   Nombre total vélos disponibles  1472 non-null   int64  
 6   Vélos mécaniques disponibles    1472 non

In [8]:
df_disp_velib.info()
display(df_disp_velib.head())
df_cpt_velib_desc = df_disp_velib.select_dtypes(include=np.number).describe()
display(df_cpt_velib_desc)
df_cpt_velib_cr = df_disp_velib.select_dtypes(include=np.number).corr()
display(df_cpt_velib_cr)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1472 entries, 0 to 1471
Data columns (total 15 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   identifiant_station             1472 non-null   object 
 1   nom_station                     1472 non-null   object 
 2   station_en_fonctionnement       1472 non-null   object 
 3   capacite_de_la_station          1472 non-null   int64  
 4   nombre_bornettes_libres         1472 non-null   int64  
 5   nombre_total_velos_disponibles  1472 non-null   int64  
 6   velos_mecaniques_disponibles    1472 non-null   int64  
 7   velos_electriques_disponibles   1472 non-null   int64  
 8   borne_de_paiement_disponible    1472 non-null   object 
 9   retour_velib_possible           1472 non-null   object 
 10  actualisation_de_la_donnee      1472 non-null   object 
 11  coordonnees_geographiques       1472 non-null   object 
 12  nom_communes_equipees           14

,identifiant_station,nom_station,station_en_fonctionnement,capacite_de_la_station,nombre_bornettes_libres,nombre_total_velos_disponibles,velos_mecaniques_disponibles,velos_electriques_disponibles,borne_de_paiement_disponible,retour_velib_possible,actualisation_de_la_donnee,coordonnees_geographiques,nom_communes_equipees,code_insee_communes_equipees,station_opening_hours
0,16107,Benjamin Godard - Victor Hugo,OUI,0,27,8,0,8,OUI,OUI,2025-04-17T15:39:50+00:00,"48.865983, 2.275725",Paris,75056,NaN
1,44015,Rouget de L'isle - Watteau,OUI,0,12,8,4,4,OUI,OUI,2025-04-17T15:37:57+00:00,"48.778192750803, 2.3963020229163",Vitry-sur-Seine,94081,NaN
2,40001,Hôpital Mondor,OUI,0,23,6,2,4,OUI,OUI,2025-04-17T15:37:02+00:00,"48.798922410229, 2.4537451531298",Créteil,94028,NaN
3,9020,Toudouze - Clauzel,OUI,0,18,2,0,2,OUI,OUI,2025-04-17T15:41:24+00:00,"48.87929591733507, 2.3373600840568547",Paris,75056,NaN
4,13007,Le Brun - Gobelins,OUI,0,37,10,6,4,OUI,OUI,2025-04-17T15:38:07+00:00,"48.835092787824, 2.3534681351338",Paris,75056,NaN


,capacite_de_la_station,nombre_bornettes_libres,nombre_total_velos_disponibles,velos_mecaniques_disponibles,velos_electriques_disponibles,code_insee_communes_equipees,station_opening_hours
count,1472.000000,1472.000000,1472.000000,1472.000000,1472.000000,1472.000000,0.0
mean,0.366848,20.063179,10.333560,6.832201,3.501359,80987.333560,NaN
std,2.312426,11.796055,10.603836,9.249600,3.296786,8415.144634,NaN
min,0.000000,0.000000,0.000000,0.000000,0.000000,75056.000000,NaN
25%,0.000000,11.750000,3.000000,1.000000,1.000000,75056.000000,NaN
50%,0.000000,19.000000,7.000000,3.000000,3.000000,75056.000000,NaN
75%,0.000000,27.000000,14.000000,10.000000,5.000000,92040.000000,NaN
max,50.000000,65.000000,68.000000,63.000000,23.000000,95018.000000,NaN


,capacite_de_la_station,nombre_bornettes_libres,nombre_total_velos_disponibles,velos_mecaniques_disponibles,velos_electriques_disponibles,code_insee_communes_equipees,station_opening_hours
capacite_de_la_station,1.000000,0.047723,0.005541,-0.011645,0.050495,-0.056901,NaN
nombre_bornettes_libres,0.047723,1.000000,-0.435707,-0.407120,-0.259180,-0.082124,NaN
nombre_total_velos_disponibles,0.005541,-0.435707,1.000000,0.953942,0.539997,-0.063042,NaN
velos_mecaniques_disponibles,-0.011645,-0.407120,0.953942,1.000000,0.262633,-0.043577,NaN
velos_electriques_disponibles,0.050495,-0.259180,0.539997,0.262633,1.000000,-0.080509,NaN
code_insee_communes_equipees,-0.056901,-0.082124,-0.063042,-0.043577,-0.080509,1.000000,NaN
station_opening_hours,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 2.2.2 VELIB COMPTAGE

In [ ]:
df_cpt_velib_raw = dfc.load_dataset_from_config('velib_comptage_data', sep=';')

if df_cpt_velib_raw is not None and isinstance(df_cpt_velib_raw, pd.DataFrame):
    display(df_cpt_velib_raw.head())
    dfc.log_general_info(df_cpt_velib_raw)
    nb_first, nb_total = dfc.detect_and_log_duplicates_and_missing(df_cpt_velib_raw)
    if nb_first != nb_total:
        print(dfc.duplicates_index_map(df_cpt_velib_raw))
    df_cpt_velib = dfc.normalize_column_names(df_cpt_velib_raw)

[INFO]-[2025-06-01 09:52:15,145] File path resolved from configuration : https://drive.google.com/file/d/1IN7U_BkhrfPEfjCvD61sxVo_zMsKk6A9/view?usp=sharing.
[INFO]-[2025-06-01 09:52:15,148] File ID extracted from URL: 1IN7U_BkhrfPEfjCvD61sxVo_zMsKk6A9
[INFO]-[2025-06-01 09:52:15,535] Big file mechanism : Attempting download from confirmation URL.


,Identifiant du compteur,Nom du compteur,Identifiant du site de comptage,Nom du site de comptage,Comptage horaire,Date et heure de comptage,Date d'installation du site de comptage,Lien vers photo du site de comptage,Coordonnées géographiques,Identifiant technique compteur,ID Photos,test_lien_vers_photos_du_site_de_comptage_,id_photo_1,url_sites,type_dimage,mois_annee_comptage
0,100003098-101003098,106 avenue Denfert Rochereau NE-SO,100003098.0,106 avenue Denfert Rochereau,0,2024-03-01T06:00:00+01:00,2012-02-22,https://filer.eco-counter-tools.com/file/09/73...,"48.83507,2.33305",Y2H20114504,https://filer.eco-counter-tools.com/file/09/73...,https://filer.eco-counter-tools.com/file/09/73...,https:,https://www.eco-visio.net/Photos/100003098,jpg,2024-03
1,100003098-101003098,106 avenue Denfert Rochereau NE-SO,100003098.0,106 avenue Denfert Rochereau,0,2024-03-01T05:00:00+01:00,2012-02-22,https://filer.eco-counter-tools.com/file/09/73...,"48.83507,2.33305",Y2H20114504,https://filer.eco-counter-tools.com/file/09/73...,https://filer.eco-counter-tools.com/file/09/73...,https:,https://www.eco-visio.net/Photos/100003098,jpg,2024-03
2,100003098-101003098,106 avenue Denfert Rochereau NE-SO,100003098.0,106 avenue Denfert Rochereau,0,2024-03-01T04:00:00+01:00,2012-02-22,https://filer.eco-counter-tools.com/file/09/73...,"48.83507,2.33305",Y2H20114504,https://filer.eco-counter-tools.com/file/09/73...,https://filer.eco-counter-tools.com/file/09/73...,https:,https://www.eco-visio.net/Photos/100003098,jpg,2024-03
3,100003098-101003098,106 avenue Denfert Rochereau NE-SO,100003098.0,106 avenue Denfert Rochereau,0,2024-03-01T12:00:00+01:00,2012-02-22,https://filer.eco-counter-tools.com/file/09/73...,"48.83507,2.33305",Y2H20114504,https://filer.eco-counter-tools.com/file/09/73...,https://filer.eco-counter-tools.com/file/09/73...,https:,https://www.eco-visio.net/Photos/100003098,jpg,2024-03
4,100003098-101003098,106 avenue Denfert Rochereau NE-SO,100003098.0,106 avenue Denfert Rochereau,0,2024-03-01T08:00:00+01:00,2012-02-22,https://filer.eco-counter-tools.com/file/09/73...,"48.83507,2.33305",Y2H20114504,https://filer.eco-counter-tools.com/file/09/73...,https://filer.eco-counter-tools.com/file/09/73...,https:,https://www.eco-visio.net/Photos/100003098,jpg,2024-03


[INFO]-[2025-06-01 09:52:51,614] Dataset shape: 942554 rows x 16 columns
[INFO]-[2025-06-01 09:52:51,615] For quantitative variable description use:
df.select_dtypes(include=np.number).describe()
[INFO]-[2025-06-01 09:52:51,615] For quantitative correlation matrix use:
df.select_dtypes(include=np.number).corr()
[INFO]-[2025-06-01 09:52:52,000] DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 942554 entries, 0 to 942553
Data columns (total 16 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   Identifiant du compteur                     905020 non-null  object 
 1   Nom du compteur                             942554 non-null  object 
 2   Identifiant du site de comptage             905020 non-null  float64
 3   Nom du site de comptage                     905020 non-null  object 
 4   Comptage horaire                            942554 non-null  int64  
 5   Date

,identifiant_du_compteur,nom_du_compteur,identifiant_du_site_de_comptage,nom_du_site_de_comptage,comptage_horaire,date_et_heure_de_comptage,date_d_installation_du_site_de_comptage,lien_vers_photo_du_site_de_comptage,coordonnees_geographiques,identifiant_technique_compteur,id_photos,test_lien_vers_photos_du_site_de_comptage,id_photo_1,url_sites,type_dimage,mois_annee_comptage
0,100003098-101003098,106 avenue Denfert Rochereau NE-SO,100003098.0,106 avenue Denfert Rochereau,0,2024-03-01T06:00:00+01:00,2012-02-22,https://filer.eco-counter-tools.com/file/09/73...,"48.83507,2.33305",Y2H20114504,https://filer.eco-counter-tools.com/file/09/73...,https://filer.eco-counter-tools.com/file/09/73...,https:,https://www.eco-visio.net/Photos/100003098,jpg,2024-03
1,100003098-101003098,106 avenue Denfert Rochereau NE-SO,100003098.0,106 avenue Denfert Rochereau,0,2024-03-01T05:00:00+01:00,2012-02-22,https://filer.eco-counter-tools.com/file/09/73...,"48.83507,2.33305",Y2H20114504,https://filer.eco-counter-tools.com/file/09/73...,https://filer.eco-counter-tools.com/file/09/73...,https:,https://www.eco-visio.net/Photos/100003098,jpg,2024-03
2,100003098-101003098,106 avenue Denfert Rochereau NE-SO,100003098.0,106 avenue Denfert Rochereau,0,2024-03-01T04:00:00+01:00,2012-02-22,https://filer.eco-counter-tools.com/file/09/73...,"48.83507,2.33305",Y2H20114504,https://filer.eco-counter-tools.com/file/09/73...,https://filer.eco-counter-tools.com/file/09/73...,https:,https://www.eco-visio.net/Photos/100003098,jpg,2024-03
3,100003098-101003098,106 avenue Denfert Rochereau NE-SO,100003098.0,106 avenue Denfert Rochereau,0,2024-03-01T12:00:00+01:00,2012-02-22,https://filer.eco-counter-tools.com/file/09/73...,"48.83507,2.33305",Y2H20114504,https://filer.eco-counter-tools.com/file/09/73...,https://filer.eco-counter-tools.com/file/09/73...,https:,https://www.eco-visio.net/Photos/100003098,jpg,2024-03
4,100003098-101003098,106 avenue Denfert Rochereau NE-SO,100003098.0,106 avenue Denfert Rochereau,0,2024-03-01T08:00:00+01:00,2012-02-22,https://filer.eco-counter-tools.com/file/09/73...,"48.83507,2.33305",Y2H20114504,https://filer.eco-counter-tools.com/file/09/73...,https://filer.eco-counter-tools.com/file/09/73...,https:,https://www.eco-visio.net/Photos/100003098,jpg,2024-03


In [9]:
df_cpt_velib.info()
display(df_cpt_velib.head())
df_cpt_velib_desc = df_cpt_velib.select_dtypes(include=np.number).describe()
display(df_cpt_velib_desc)
df_cpt_velib_cr = df_cpt_velib.select_dtypes(include=np.number).corr()
display(df_cpt_velib_cr)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 942554 entries, 0 to 942553
Data columns (total 16 columns):
 #   Column                                     Non-Null Count   Dtype  
---  ------                                     --------------   -----  
 0   identifiant_du_compteur                    905020 non-null  object 
 1   nom_du_compteur                            942554 non-null  object 
 2   identifiant_du_site_de_comptage            905020 non-null  float64
 3   nom_du_site_de_comptage                    905020 non-null  object 
 4   comptage_horaire                           942554 non-null  int64  
 5   date_et_heure_de_comptage                  942554 non-null  object 
 6   date_d_installation_du_site_de_comptage    905020 non-null  object 
 7   lien_vers_photo_du_site_de_comptage        895215 non-null  object 
 8   coordonnees_geographiques                  905020 non-null  object 
 9   identifiant_technique_compteur             895215 non-null  object 
 10  id_photo

,identifiant_du_compteur,nom_du_compteur,identifiant_du_site_de_comptage,nom_du_site_de_comptage,comptage_horaire,date_et_heure_de_comptage,date_d_installation_du_site_de_comptage,lien_vers_photo_du_site_de_comptage,coordonnees_geographiques,identifiant_technique_compteur,id_photos,test_lien_vers_photos_du_site_de_comptage,id_photo_1,url_sites,type_dimage,mois_annee_comptage
0,100003098-101003098,106 avenue Denfert Rochereau NE-SO,100003098.0,106 avenue Denfert Rochereau,0,2024-03-01T06:00:00+01:00,2012-02-22,https://filer.eco-counter-tools.com/file/09/73...,"48.83507,2.33305",Y2H20114504,https://filer.eco-counter-tools.com/file/09/73...,https://filer.eco-counter-tools.com/file/09/73...,https:,https://www.eco-visio.net/Photos/100003098,jpg,2024-03
1,100003098-101003098,106 avenue Denfert Rochereau NE-SO,100003098.0,106 avenue Denfert Rochereau,0,2024-03-01T05:00:00+01:00,2012-02-22,https://filer.eco-counter-tools.com/file/09/73...,"48.83507,2.33305",Y2H20114504,https://filer.eco-counter-tools.com/file/09/73...,https://filer.eco-counter-tools.com/file/09/73...,https:,https://www.eco-visio.net/Photos/100003098,jpg,2024-03
2,100003098-101003098,106 avenue Denfert Rochereau NE-SO,100003098.0,106 avenue Denfert Rochereau,0,2024-03-01T04:00:00+01:00,2012-02-22,https://filer.eco-counter-tools.com/file/09/73...,"48.83507,2.33305",Y2H20114504,https://filer.eco-counter-tools.com/file/09/73...,https://filer.eco-counter-tools.com/file/09/73...,https:,https://www.eco-visio.net/Photos/100003098,jpg,2024-03
3,100003098-101003098,106 avenue Denfert Rochereau NE-SO,100003098.0,106 avenue Denfert Rochereau,0,2024-03-01T12:00:00+01:00,2012-02-22,https://filer.eco-counter-tools.com/file/09/73...,"48.83507,2.33305",Y2H20114504,https://filer.eco-counter-tools.com/file/09/73...,https://filer.eco-counter-tools.com/file/09/73...,https:,https://www.eco-visio.net/Photos/100003098,jpg,2024-03
4,100003098-101003098,106 avenue Denfert Rochereau NE-SO,100003098.0,106 avenue Denfert Rochereau,0,2024-03-01T08:00:00+01:00,2012-02-22,https://filer.eco-counter-tools.com/file/09/73...,"48.83507,2.33305",Y2H20114504,https://filer.eco-counter-tools.com/file/09/73...,https://filer.eco-counter-tools.com/file/09/73...,https:,https://www.eco-visio.net/Photos/100003098,jpg,2024-03


,identifiant_du_site_de_comptage,comptage_horaire
count,9.050200e+05,942554.000000
mean,1.348171e+08,77.879018
std,7.579019e+07,106.928569
min,1.000031e+08,0.000000
25%,1.000475e+08,11.000000
50%,1.000560e+08,42.000000
75%,1.000563e+08,97.000000
max,3.000303e+08,3070.000000


,identifiant_du_site_de_comptage,comptage_horaire
identifiant_du_site_de_comptage,1.000000,-0.119705
comptage_horaire,-0.119705,1.000000


## 2.2 Data quality refinement

In [7]:
# Original backup and duplicates management
df_disp_velib_orig = df_disp_velib.copy()
df_disp_velib = df_disp_velib.drop_duplicates()
df_cpt_velib_orig = df_cpt_velib.copy()
df_cpt_velib = df_cpt_velib.drop_duplicates()

## 2.3 Data combination and rework

# 2. Data Viz' and Analysis

## 2.1 General Data Viz'

## 2.2 Quantitative mono variable distribution

## 2.3 Qualitative mono variable distribution

## 2.4 Qualitative multi variable distribution

## 2.5 Quantitative multi variable correlation